Location of impedance modules: C:\Users\andba122\AppData\Local\anaconda3\Lib\site-packages\impedance

In [138]:
## OPEN IDFs, SAVE AS CSVs

## Import Ivium data and convert to CSV for entire folder

import glob
import numpy as np
import os

folder_address = '.'    # current directory

# Find all files ending in '.idf' and list their filenames
idf_files = glob.glob('*.idf')
print(idf_files)

# Load data from the example EIS data; 
# INSERT FILE NAME: (Format needs to be:  Frequency, Real, Imaginary)

# Process each file in the idf_files list
for current_file in range(len(idf_files)):
    file_name = idf_files[current_file] if idf_files else "No files found"
    print(f"Using file: {file_name}")


    # Read file content
    with open(file_name, "r", errors="ignore") as file:
        lines = file.readlines()

    # Locate the numerical data section
    data_start = None
    for i, line in enumerate(lines):
        if line.strip().isdigit():  # Identifies the "71" line before data
            data_start = i + 1
            break

    # Extract and parse numerical data
    data_lines = lines[data_start : data_start + 71]  # Adjust row count if needed
    data = np.array([list(map(float, line.split())) for line in data_lines])

    # Rearrange columns: Frequency first, Z_real second, Z_imaginary third
    reordered_data = data[:, [2, 0, 1]]

    # Generate the CSV file name by replacing the .idf extension with .csv
    csv_file_name = os.path.splitext(file_name)[0] + '.csv'

    # Save the extracted data
    np.savetxt(csv_file_name, reordered_data, delimiter=",", comments='')

    print(f"Data extracted and saved as '{csv_file_name}'")


# Find all files ending in '.csv' and list their filenames
csv_files = glob.glob('*.csv')
print(csv_files)



['0421_250212EISScan6_B59313_8143,7b.idf', '0422_250212EISScan7_B59313_8143,7b2.idf', '0522_250212EISScan1_B59313_EIS1.idf', '0616_250213EISScan1_B59313_EIS1.idf', '0710_250213EISScan1_B59313_EIS1.idf', '0804_250213EISScan1_B59313_EIS1.idf', '0898_250213EISScan1_B59313_EIS1.idf', '0992_250213EISScan1_B59313_EIS1.idf', '1086_250213EISScan1_B59313_EIS1.idf', '1180_250213EISScan1_B59313_EIS1.idf', '1274_250213EISScan1_B59313_EIS1.idf', '1368_250213EISScan1_B59313_EIS1.idf']
Using file: 0421_250212EISScan6_B59313_8143,7b.idf
Data extracted and saved as '0421_250212EISScan6_B59313_8143,7b.csv'
Using file: 0422_250212EISScan7_B59313_8143,7b2.idf
Data extracted and saved as '0422_250212EISScan7_B59313_8143,7b2.csv'
Using file: 0522_250212EISScan1_B59313_EIS1.idf
Data extracted and saved as '0522_250212EISScan1_B59313_EIS1.csv'
Using file: 0616_250213EISScan1_B59313_EIS1.idf
Data extracted and saved as '0616_250213EISScan1_B59313_EIS1.csv'
Using file: 0710_250213EISScan1_B59313_EIS1.idf
Data e

In [139]:
# To enable concatentation of all fitting data for all circuits of this dataset - creates empty dataframe (WILL WIPE PREVIOUS DATAFRAME)
# import pandas as pd
# results_expt_all_circs_df = pd.DataFrame()



In [140]:
# DEFINES CUSTOM CIRCUITS USING THE IMPEDANCE.MODELS.CIRCUITS MODULE

from impedance.models.circuits import CustomCircuit

from impedance.models.circuits.elements import element
import numpy as np

film_initial_thickness = 4e-9   # INPUT:  __ nm ALD film starting thickness

solution_resistance_min = 120   # INPUT:  __ Ohm minimum solution resistance
solution_resistance_max = 180  # INPUT:  __ Ohm maximum solution resistance
CPE_alpha_min = 0.65            # INPUT:  __ minimum CPE alpha value

rho_0_guess = 1e10 # INPUT: Initial guess for the resistivity of the film (Ohm m)
R_1_guess = 1000 # INPUT: Initial guess for the charge transfer resistance (Ohm)


# Defines the custom element for a Young Impedance with exponentially decreasing resistivity
@element(4, units=["Ohm m", "m", "F/m", "m"], overwrite=True)
def CustomR(p, f):

    # Permittivity = _____  (to remove permittivity as a degree of freedom, set to typical value for Al2O3)
    # 9 * 8.8541878188 * 10**-12 F/m ~8e-11 F/m for typical Al2O3   
    rho_0, lambda_param, epsilonepsilon_0, delta = p
    omega = 2 * np.pi * np.array(f)       

    numerator = 1 + 1j * omega * epsilonepsilon_0 * rho_0 * np.exp(-delta / lambda_param)
    denominator = 1 + 1j * omega * epsilonepsilon_0 * rho_0
    Z_x_area = (-lambda_param / (1j * omega * epsilonepsilon_0)) * np.log(numerator / denominator)
    area = 1.9635e-5   # m2

    Z = Z_x_area / area
    return Z

    #NOTE CHECK THIS ALGEB/RAIC FORMULATION OF YOUNG IMPEDANCE (IS IT CORRECT?)


# Create a list of circuits and default initial guesses for fitting
equivalence_circuits = []
initial_guesses = []
parameter_strings = []
bounds_defaults = []


#N.B. MAY HAVE TO ADD ADDITIONAL CIRCUITS WITHOUT WARBURG OR WITH OTHER ADJUSTMENTS (SEE IF NECESSARY IN IVIUM FITTING)


# Define the 0th circuit (Randles)
equivalence_circuits.append("R0-p(R1-W1,C1)")
initial_guesses.append([150, 1000, 1000, 2e-6])
parameter_strings.append("R0 (Ohm), R1 (Ohm), W1 (Ohm/sqrt(s)), C1 (F)")
bounds_defaults.append((
    [solution_resistance_min, 0, 0, 0],                           # Lower bounds
    [solution_resistance_max, np.inf, np.inf, np.inf], ))    # Upper bounds


# Define the 1st circuit (Randles with constant phase element)
equivalence_circuits.append("R0-p(R1-W1,CPE1)")
initial_guesses.append([150, 1000, 1000, 2e-6, 1])
parameter_strings.append("R0 (Ohm), R1 (Ohm), W1 (Ohm/sqrt(s)), Q1 (F), a1 (0-1)")
bounds_defaults.append((
    [solution_resistance_min, 0, 0, 0, CPE_alpha_min],                           # Lower bounds
    [solution_resistance_max, np.inf, np.inf, np.inf, 1], ))    # Upper bounds


# Define the 2nd circuit (Film + Randles with CPEs)
equivalence_circuits.append("R0-p(CPE1,R1-p(R2-W1,CPE2))")
initial_guesses.append([150, 2e-9, 1, 1000, 1000, 1000, 2e-6, 1])
parameter_strings.append("R0 (Ohm), Q1 (F), a1 (0-1), R1 (Ohm), R2 (Ohm), W1 (Ohm/sqrt(s)), Q2 (F), a2 (0-1)")
bounds_defaults.append((
    [solution_resistance_min, 0, CPE_alpha_min, 0, 0, 0, 0, CPE_alpha_min],                           # Lower bounds
    [solution_resistance_max, np.inf, 1, np.inf, np.inf, np.inf, np.inf, 1], ))    # Upper bounds


# Define the 3rd circuit (Randles, no Warburg for diffusion)
equivalence_circuits.append("R0-p(R1,C1)")
initial_guesses.append([150, 1000, 2e-6])
parameter_strings.append("R0 (Ohm), R1 (Ohm), C1 (F)")
bounds_defaults.append((
    [solution_resistance_min, 0, 0],                           # Lower bounds
    [solution_resistance_max, np.inf, np.inf], ))    # Upper bounds


# Define the 4th circuit (Randles with constant phase element, no Warburg for diffusion)
equivalence_circuits.append("R0-p(R1,CPE1)")
initial_guesses.append([150, 1000, 2e-6, 1])
parameter_strings.append("R0 (Ohm), R1 (Ohm), Q1 (F), a1 (0-1)")
bounds_defaults.append((
    [solution_resistance_min, 0, 0, CPE_alpha_min],                           # Lower bounds
    [solution_resistance_max, np.inf, np.inf, 1], ))    # Upper bounds


# Define the 5th circuit (Film + Randles with CPEs, no Warburg for diffusion)
equivalence_circuits.append("R0-p(CPE1,R1-p(R2,CPE2))")
initial_guesses.append([150, 2e-9, 1, 1000, 1000, 2e-6, 1])
parameter_strings.append("R0 (Ohm), Q1 (F), a1 (0-1), R1 (Ohm), R2 (Ohm), Q2 (F), a2 (0-1)")
bounds_defaults.append((
    [solution_resistance_min, 0, CPE_alpha_min, 0, 0, 0, CPE_alpha_min],                           # Lower bounds
    [solution_resistance_max, np.inf, 1, np.inf, np.inf, np.inf, 1], ))    # Upper bounds


## EXPERIMENTAL CIRCUITS CONTAINING YOUNG IMPEDANCE:
# NOTE: 
# [Circ. 6/7: In literature, Haeffele et al., but not sure this makes sense - Should there be a pore resistance in parallel to the Young impedance?/Should the Young impedance be in series with the Randles?]
# In principle, the ratio of lambda to delta should be related (?). Seems to typically be lambda = 1/5 delta in Haeffele et al.
# Delta should be limited to similar order of magnitude as original film thickness,
# but seems to grow a bit in Haeffele et al's fitting (restructuration possible?)
# It seems possible to extract both, since lambda is also into the pre-exponent
# Rho maybe should be fixed to typical physical values for amorphous Al2O3 as well
# Set delta default to film thickness...

# Define the 6th circuit (Young Impedance parallel to Randles with CPE, c.f. Haeffele et al.) 
equivalence_circuits.append("R0-p(CustomR1,R1-W1,CPE1)")
initial_guesses.append([150, rho_0_guess, film_initial_thickness/5, 7.9686e-11, film_initial_thickness, R_1_guess, 1000, 2e-6, 1])
parameter_strings.append("R0 (Ohm), Rho_0 (Ohm m), lambda_param (m), epsilonepsilon_0 (F/m), delta (m), R1 (Ohm), W1 (Ohm/sqrt(s)), Q1 (F), a (0-1)")
# Permittivity (9 * 8.8541878188 * 10**-12 F/m ~8e-11 F/m) for typical Al2O3
bounds_defaults.append((
    [solution_resistance_min, 0, 0, 7.9685e-11, 0, 0, 0, 0, CPE_alpha_min],                           # Lower bounds
    [solution_resistance_max, np.inf, np.inf, 7.9687e-11, (film_initial_thickness*10), np.inf, np.inf, np.inf, 1], ))    # Upper bounds


# Define the 7th circuit (Young Impedance parallel to Randles with CPE, Warburg in series with Young Impedance)
equivalence_circuits.append("R0-p(CustomR1-W1,R1,CPE1)")
initial_guesses.append([150, rho_0_guess, film_initial_thickness/5, 7.9685e-11, film_initial_thickness, 1000, R_1_guess, 2e-6, 1])
parameter_strings.append("R0 (Ohm), Rho_0 (Ohm m), lambda_param (m), epsilonepsilon_0 (F/m), delta (m), W1 (Ohm/sqrt(s)), R1 (Ohm), Q1 (F), a1 (0-1)")
bounds_defaults.append((
    [0, 0, 0, 7.9685e-11, 0, 0, 0, 0, 0.55],                           # Lower bounds
    [np.inf, np.inf, np.inf, 7.9687e-11, (film_initial_thickness*10), np.inf, np.inf, np.inf, 1], ))    # Upper bounds


# Define the 8th circuit (Young Impedance parallel to Randles with CPE, NO Warburg for diffusion) 
equivalence_circuits.append("R0-p(CustomR1,R1,CPE1)")
initial_guesses.append([150, rho_0_guess, film_initial_thickness/5, 7.9686e-11, film_initial_thickness, R_1_guess, 2e-6, 1])
parameter_strings.append("R0 (Ohm), Rho_0 (Ohm m), lambda_param (m), epsilonepsilon_0 (F/m), delta (m), R1 (Ohm), Q1 (F), a (0-1)")
# Permittivity (9 * 8.8541878188 * 10**-12 F/m ~8e-11 F/m) for typical Al2O3  ---  See previous NOTES above for circuit 3!
bounds_defaults.append((
    [0, 0, 0, 7.9685e-11, 0, 0, 0, CPE_alpha_min],                           # Lower bounds
    [np.inf, np.inf, np.inf, 7.9687e-11, (film_initial_thickness*10), np.inf, np.inf, 1], ))    # Upper bounds


# Define the 9th circuit (Young Impedance parallel to Randles with CPE, Warburg in series with BOTH Z_Y and R_CT)
equivalence_circuits.append("R0-p(p(R1,CustomR1)-W1,CPE1)")     #   "R0-p(p(CustomR1,R1)-W1),CPE1" -- does not work ?
initial_guesses.append([150, 1000, rho_0_guess, film_initial_thickness/10, 7.9685e-11, film_initial_thickness, 1000, 2e-6, 1])
parameter_strings.append("R0 (Ohm), R1 (Ohm), Rho_0 (Ohm m), lambda_param (m), epsilonepsilon_0 (F/m), delta (m), W1 (Ohm/sqrt(s)), Q1 (F), a1 (0-1)")
bounds_defaults.append((
    [0, 0, 0, 0, 7.9685e-11, 0, 0, 0, CPE_alpha_min],                           # Lower bounds
    [np.inf, np.inf, np.inf, np.inf, 7.9687e-11, (film_initial_thickness*10), np.inf, np.inf, 1], ))    # Upper bounds


# Define the 10th circuit (Young Impedance in series before Randles with CPE, Warburg in series with R_CT)   
# -- SEEMS TO BE THE MOST PROMISING!!!
equivalence_circuits.append("R0-CustomR1-p(R1-W1,CPE1)")
initial_guesses.append([150, rho_0_guess, film_initial_thickness/5, 7.9686e-11, film_initial_thickness, R_1_guess, 1000, 2e-6, 1])
parameter_strings.append("R0 (Ohm), Rho_0 (Ohm m), lambda_param (m), epsilonepsilon_0 (F/m), delta (m), R1 (Ohm), W1 (Ohm/sqrt(s)), Q1 (F), a1 (0-1)")
bounds_defaults.append((
    [solution_resistance_min, 0, 0, 7.9685e-11, 0, 0, 0, 0, CPE_alpha_min],                           # Lower bounds
    [solution_resistance_max, np.inf, np.inf, 7.9687e-11, (film_initial_thickness*10), np.inf, np.inf, np.inf, 1], ))    # Upper bounds


# Define the 11th circuit (In series Z_Y + W) 
equivalence_circuits.append("R0-CustomR1-W1")
initial_guesses.append([150, rho_0_guess, film_initial_thickness/5, 7.9686e-11, film_initial_thickness, 1000])
parameter_strings.append("R0 (Ohm), Rho_0 (Ohm m), lambda_param (m), epsilonepsilon_0 (F/m), delta (m), W1 (Ohm/sqrt(s)")
bounds_defaults.append((
    [0, 0, 0, 7.9685e-11, 0, 0],                           # Lower bounds
    [np.inf, np.inf, np.inf, 7.9687e-11, (film_initial_thickness*10), np.inf], ))    # Upper bounds

## ADD MORE CIRCUITS HERE AS NECESSARY... 
## E.g. -- more appropriate Young impedance model may be:   R0-CustomR1-W1  or  R0-p(R1-W1,CustomR1) (?)




In [141]:
## PREPROCESSING

## Does Impedance.py preprocessing of data from CSVs -- (INSERT NO. OF FILE TO PROCESS + FITTING FREQUENCY RANGE + INITIAL FILM THICKNESS)

from impedance import preprocessing

current_file = 11        # MANUALLY CHOOSE index of the current file being processed:  Index: 0 to len(idf_files) - 1
print(f"Select from: {[0]} to {[len(idf_files) - 1]}. {[current_file]} selected.")
print(f"Using file: {csv_files[current_file]}")

file_name = csv_files[current_file]

frequencies_orig, Z_orig = preprocessing.readCSV(file_name)     

# Crop the data to only include the impedance data across selected frequencies:
# SELECTED FREQUENCIES (e.g. 10 kHz to 10 mHz i.e. 10,000 to 0.01 Hz)
freqmin = 0.01 # Hz
freqmax = 6000 # Hz
# NOTE: FITTING MAY BENEFIT FROM CUTTING RANGE TO 5.1 kHz - 0.1 Hz!)
frequencies, Z = preprocessing.cropFrequencies(frequencies_orig, Z_orig, freqmin, freqmax)

# keep only the impedance data in the first quadrant
frequencies, Z = preprocessing.ignoreBelowX(frequencies, Z)



Select from: [0] to [11]. [11] selected.
Using file: 1368_250213EISScan1_B59313_EIS1.csv


In [142]:
# FITS THE CIRCUIT TO THE DATA AND GIVES THE PARAMETERS

import pandas as pd

#circuit_to_fit = 0   #Choose from 0, 1, 2, 3...  (Old manual method)

# To enable concatentation of all fitting data for all circuits of this dataset - creates empty dataframe (WILL WIPE PREVIOUS DATAFRAME)
results_expt_all_circs_df = pd.DataFrame()
results_expt_all_circs_abbrev_df = pd.DataFrame()


# Loop over each circuit to fit
for circuit_to_fit in range(len(equivalence_circuits)):    # Turns into a loop. Adjust for desired circuits to fit.
    try:
        initial_guess_to_fit = initial_guesses[circuit_to_fit]

        #initial_guesses[circuit_to_fit]  # Can manually replace this with a list of initial guesses

        circuit = CustomCircuit(equivalence_circuits[circuit_to_fit], 
                                initial_guess=initial_guess_to_fit,)              # Initial guesses can be given here, but not bounds! (Must be given in .fit)

        # To give manual bounds, set to 1. You will have to run as single circuit at a time. (Set circuit_to_fits = 0, 1, 2 etc.)
        manual_bounds_y_n = 0                           
        if manual_bounds_y_n == 1:
            manual_bounds =     (([0, 0, 0, 0, 0, 0, 0],                        # Lower bounds
            [np.inf, np.inf, 1, np.inf, np.inf, np.inf, 1], ))                  # Upper bounds
        else: bounds_selected = bounds_defaults[circuit_to_fit]

        # circuit.fit(frequencies, Z,weight_by_modulus=True, bounds = bounds_selected)                    
        # Bounds=bounds_selected, etc. Disable for "boundless" fitting - will default to 0 - inf except for CPE exponents
        # N.B. weight_by_modulus=True is recommended for EIS data fitting for potentiostatic measurements    
        circuit.fit(frequencies, Z, weight_by_modulus=True, bounds=bounds_selected,  maxfev=10000000)

        print("Circuit",circuit_to_fit,"parameters:")
        #print(circuit.parameters_)
        #print("\nConfidence intervals for the parameters:")
        #print(circuit.conf_)
        fit_results = circuit.parameters_
        fit_results_uncert = circuit.conf_    # One standard deviation error estimates for fit parameters
        fit_results_uncertpct = 100 * fit_results_uncert / fit_results

       # fit_results_iterations = circuit.maxfev
       # print(f"Number of function evaluations: {fit_results_iterations}")
        #print("\nConfidence intervals as percentages:")


        # Create a DataFrame to concatenate the results
        labels = []
        labels = (parameter_strings[circuit_to_fit].split(", "))

        data = {
            'Fitted Value': fit_results,
            'Uncertainty': fit_results_uncert,
            'Uncertainty (%)': fit_results_uncertpct
        }

        results_df = pd.DataFrame(data, index=labels)

        # Print the DataFrame   (Hidden because repeated later)
        #print("\nExportable table of fit results:")
        #print(results_df, "\n")

        # USE IMPEDANCEPY'S INTERACTIVE PLOT TO VISUALIZE THE FIT, AND SAVE THE INTERACTIVE GRAPH

        import pandas as pd
        import matplotlib.pyplot as plt
        import warnings
        warnings.filterwarnings("ignore", category=RuntimeWarning)
        warnings.filterwarnings("ignore", category=FutureWarning)

        # Create the "fitting" subfolder if it doesn't exist
        fitting_folder = os.path.join(os.path.dirname(file_name), 'Fitting')
        os.makedirs(fitting_folder, exist_ok=True)

        # Predicts the impedance of the fitted circuit (Z1, Z2)
        Z_fit = circuit.predict(frequencies)
        # Plot the data using Matplotlib
        chart = circuit.plot(f_data=frequencies, Z_data=Z)
        # Save the plot as an interactive HTML file and a JSON file to the "fitting" subfolder
        output_chart_name_html = os.path.join(fitting_folder, file_name.replace('.csv', f'_Circ{circuit_to_fit}.html'))
        chart.save(output_chart_name_html)
        output_chart_name_json = os.path.join(fitting_folder, file_name.replace('.csv', f'_Circ{circuit_to_fit}.json'))
        chart.save(output_chart_name_json)
        #chart.display()  # Hidden because other chart contains all data

        # Same, but simulated over full freq. range
        Z_fit_full = circuit.predict(frequencies_orig)
        # Plot the data using Matplotlib
        chart = circuit.plot(f_data=frequencies_orig, Z_data=Z_orig)
        # Save the plot as an interactive HTML file and a JSON file to the "fitting" subfolder
        output_chart_name_html = os.path.join(fitting_folder, file_name.replace('.csv', f'_Circ{circuit_to_fit}full.html'))
        chart.save(output_chart_name_html)
        output_chart_name_json = os.path.join(fitting_folder, file_name.replace('.csv', f'_Circ{circuit_to_fit}full.json'))
        chart.save(output_chart_name_json)
        chart.display()
        # PREPARE DATA FOR EXPORT AND NYQUIST / BODE PLOT [EXPORTS PLOTTABLE DATA]

        import os
        import pandas as pd
        import numpy as np

        # Split the complex numbers into real and imaginary components
        Z_real = np.real(Z_orig)
        Z_imag = np.imag(Z_orig)
        Z_fit_real = np.real(Z_fit_full)
        Z_fit_imag = np.imag(Z_fit_full)
        # Calculate magnitude and phase for initial Z data
        Z_magnitude = np.abs(Z_orig)
        Z_phase = np.angle(Z_orig, deg=True)
        # Calculate magnitude and phase for fitted Z data
        Z_fit_magnitude = np.abs(Z_fit_full)
        Z_fit_phase = np.angle(Z_fit_full, deg=True)

        # Create a DataFrame with the frequencies and the real and imaginary parts of Z and Z_fit
        data_Nyq = {
            'Frequency (Hz)': frequencies_orig,
            'Z_real': Z_real,
            'Z_imag': Z_imag,
            'Z_fit_real': Z_fit_real,
            'Z_fit_imag': Z_fit_imag
        }
        df_Nyq = pd.DataFrame(data_Nyq)
        #print(data_Nyq)    # Na

        # Do the same with the magnitude and phase data for a Bode plot
        data_Bode = {
            'Frequency (Hz)': frequencies_orig,
            'Z_magnitude': Z_magnitude,
            'Z_phase': Z_phase,
            'Z_fit_magnitude': Z_fit_magnitude,
            'Z_fit_phase': Z_fit_phase
        }
        df_Bode = pd.DataFrame(data_Bode)
        #print(data_Bode)    # Na

        # Create the "fitting" subfolder if it doesn't exist
        fitting_folder = os.path.join(os.path.dirname(file_name), 'Fitting')
        os.makedirs(fitting_folder, exist_ok=True)

        # Export the Nyquist data to the "fitting" subfolder
        output_file_name = os.path.join(fitting_folder, file_name.replace('.csv', f'_pyfit_resNyqCirc{circuit_to_fit}.csv'))
        df_Nyq.to_csv(output_file_name, index=False)

        # Export the Bode data to the "fitting" subfolder
        output_file_name = os.path.join(fitting_folder, file_name.replace('.csv', f'_pyfit_resBodeCirc{circuit_to_fit}.csv'))
        df_Bode.to_csv(output_file_name, index=False)

        # CALCULATE CHI-SQUARED VALUES   (ONLY FOR THE FITTED FREQ. RANGE!)

        # Ivium normalised the values from each datapoint by dividing by the sum of the squares of the measured data. This need to be adjusted to be comparable!
        #   https://en.wikipedia.org/wiki/Reduced_chi-squared_statistic
        #   https://en.wikipedia.org/wiki/Weighted_least_squares
        #   Strictly, maybe should have x^2/v  be:  residual vector matrix transpose . weight matrix . residual vector matrix / DOF

        # # Calculate residuals for the fitted freq. range
        residuals = Z - Z_fit

        # Estimate sigma as the standard deviation of the residuals
        sigma_real = np.std(residuals.real)
        sigma_imag = np.std(residuals.imag)

        # Calculate chi-squared value
        #chi_squared_x_DOF = np.sum((residuals.real / sigma_real) ** 2 + (residuals.imag / sigma_imag) ** 2)
        #print(f"Chi-squared x DOF: {chi_squared_x_DOF}")

        # Calculate chi-squared value (rewritten to match Ivium, divided by DOF)
        chi_squared_w_DOF = np.sum( (((residuals.real)**2)+((residuals.imag)**2))/(((Z.real)**2)+((Z.imag)**2)) )
        print(f"Chi-squared w DOF: {chi_squared_w_DOF}")

        degrees_of_freedom = len(Z) - len(fit_results)   # Note: IVIUM does not take into account len(fit_results) for their DOF
        print(f"DOF: {degrees_of_freedom}")

        chi_squared_over_DOF = chi_squared_w_DOF / degrees_of_freedom
        print(f"Chi-squared divided by DOF: {chi_squared_over_DOF}")
        print(f"(This seems to match IVIUM's reduced chi-squared value in order of magnitude, but their DOF does not take into account the number of fit parameters)")

        chi_squared_not_normalised_w_DOF = np.sum( (((residuals.real)**2)+((residuals.imag)**2)) )
        print(f"Chi-squared not normalised w DOF: {chi_squared_not_normalised_w_DOF}")

        chi_squared_not_normalised_over_DOF = chi_squared_not_normalised_w_DOF / degrees_of_freedom
        print(f"Chi-squared not normalised divided by DOF: {chi_squared_not_normalised_over_DOF}")

                # Print residuals
        #print("Residuals (real part):", residuals.real)
        #print("Residuals (imaginary part):", residuals.imag)


        # CODE TO APPEND CHI SQ TO RESULTS OUTPUT AND EXPORT!

        import pandas as pd
        import numpy as np

        # Append chi-squared values to the DataFrame
        chi_squared_data = pd.DataFrame({
            'Fitted Value': [equivalence_circuits[circuit_to_fit], freqmin, freqmax, chi_squared_w_DOF, chi_squared_over_DOF, ],
            'Uncertainty': [np.nan, np.nan, np.nan, np.nan, np.nan],
            'Uncertainty (%)': [np.nan, np.nan, np.nan, np.nan, np.nan]
        }, index=[f"Circuit {circuit_to_fit}", 'Freqmin (Hz)', 'Freqmax (Hz)', 'Chi-squared', 'Chi-squared/DOF'])

        spacer_df = pd.DataFrame({'Fitted Value': [np.nan],'Uncertainty': [np.nan], 'Uncertainty (%)': [np.nan]}, index=['',])

        # Concatenate the results DataFrame with the chi-squared DataFrame
        results_expt_df = pd.concat([chi_squared_data, results_df, spacer_df])

        # Print the DataFrame
        print("\nExportable table of circuit",circuit_to_fit,"fit results:")
        print(chi_squared_data, "\n")
        #print(results_expt_df)
        print(results_df, "\n")

        # Create the "fitting" subfolder if it doesn't exist
        fitting_folder = os.path.join(os.path.dirname(file_name), 'Fitting')
        os.makedirs(fitting_folder, exist_ok=True)

        # Export the DataFrame to a CSV file in the "fitting" subfolder
        output_file_name3 = os.path.join(fitting_folder, file_name.replace('.csv', f'_pyfit_FitParamCirc{circuit_to_fit}.csv'))
        results_expt_df.to_csv(output_file_name3, index_label='Parameter')

        import os
        import pandas as pd
        import numpy as np

        # Create the "fitting" subfolder if it doesn't exist
        fitting_folder = os.path.join(os.path.dirname(file_name), 'Fitting')
        os.makedirs(fitting_folder, exist_ok=True)

        # Export the DataFrame to a CSV file in the "fitting" subfolder
        output_file_name3 = os.path.join(fitting_folder, file_name.replace('.csv', f'_pyfit_FitParamCirc{circuit_to_fit}.csv'))
        results_expt_df.to_csv(output_file_name3, index_label='Parameter')
        
        # Concatenate the results DataFrame with the all circuits DataFrame
        results_expt_all_circs_df = pd.concat([results_expt_all_circs_df, results_expt_df])


        # Make an abbreviated dataFrame for the chi-squared values for rapid comparison
        chi_squared_data_abbrev_df = pd.DataFrame({

            'Chi^2': [chi_squared_w_DOF],
            'Chi^2/DOF': [chi_squared_over_DOF],
        }, index=[f"Circuit {circuit_to_fit} {equivalence_circuits[circuit_to_fit]}"])

        results_expt_all_circs_abbrev_df = pd.concat([results_expt_all_circs_abbrev_df, chi_squared_data_abbrev_df])
                
    except Exception as e:
        print(f"\nWarning: Circuit {circuit_to_fit} fitting failed with error: {str(e)}")
        print(f"Skipping circuit {circuit_to_fit} and continuing with next circuit...")
        continue

# Export the final results outside the loop
if not results_expt_all_circs_df.empty:

    # Export the DataFrame with all circuits to a CSV file in the "fitting" subfolder
    output_file_name4 = os.path.join(fitting_folder, '_' + file_name.replace('.csv', f'_pyfit_FitParamAllCircs.csv'))
    results_expt_all_circs_df.to_csv(output_file_name4, index_label='Parameter')

    # Export the Abbreviated DataFrame with just Chi_Sq to a CSV file in the "fitting" subfolder
    # Generate the export file name using current_file and the first 4 characters of file_name
    export_file_prefix = f"_EIS_Scan_{current_file}_{file_name[:4]}"
    output_file_name5 = os.path.join(fitting_folder, f"{export_file_prefix}_shortChiSqs.csv")
    results_expt_all_circs_abbrev_df.to_csv(output_file_name5, index_label=[f"EIS Scan {current_file}"])



Circuit 0 parameters:


c:\Users\andba122\AppData\Local\anaconda3\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\andba122\AppData\Local\anaconda3\Lib\site-packages\altair\vegalite\v5\api.py:362: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(
c:\Users\andba122\AppData\Local\anaconda3\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\andba122\AppData\Local\anaconda3\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\andba122\AppData\Local\anaconda3\Lib\site-packages\altair\vegalite\v5\api.py:362: AltairDep

alt.HConcatChart(...)

Chi-squared w DOF: 0.03712138263944043
DOF: 54
Chi-squared divided by DOF: 0.0006874330118414895
(This seems to match IVIUM's reduced chi-squared value in order of magnitude, but their DOF does not take into account the number of fit parameters)
Chi-squared not normalised w DOF: 191725.35356156545
Chi-squared not normalised divided by DOF: 3550.4695103993604

Exportable table of circuit 0 fit results:
                   Fitted Value  Uncertainty  Uncertainty (%)
Circuit 0        R0-p(R1-W1,C1)          NaN              NaN
Freqmin (Hz)               0.01          NaN              NaN
Freqmax (Hz)               6000          NaN              NaN
Chi-squared            0.037121          NaN              NaN
Chi-squared/DOF        0.000687          NaN              NaN 

                  Fitted Value   Uncertainty  Uncertainty (%)
R0 (Ohm)            134.461179  8.537771e-01         0.634962
R1 (Ohm)           1493.671212  7.804524e+00         0.522506
W1 (Ohm/sqrt(s))   1454.553485  9.2

c:\Users\andba122\AppData\Local\anaconda3\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\andba122\AppData\Local\anaconda3\Lib\site-packages\altair\vegalite\v5\api.py:362: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(
c:\Users\andba122\AppData\Local\anaconda3\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\andba122\AppData\Local\anaconda3\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\andba122\AppData\Local\anaconda3\Lib\site-packages\altair\vegalite\v5\api.py:362: AltairDep

...

alt.HConcatChart(...)

Chi-squared w DOF: 15.200396312278222
DOF: 49
Chi-squared divided by DOF: 0.3102121696383311
(This seems to match IVIUM's reduced chi-squared value in order of magnitude, but their DOF does not take into account the number of fit parameters)
Chi-squared not normalised w DOF: 162213836.78944737
Chi-squared not normalised divided by DOF: 3310486.465090763

Exportable table of circuit 10 fit results:
                              Fitted Value  Uncertainty  Uncertainty (%)
Circuit 10       R0-CustomR1-p(R1-W1,CPE1)          NaN              NaN
Freqmin (Hz)                          0.01          NaN              NaN
Freqmax (Hz)                          6000          NaN              NaN
Chi-squared                      15.200396          NaN              NaN
Chi-squared/DOF                   0.310212          NaN              NaN 

                        Fitted Value   Uncertainty  Uncertainty (%)
R0 (Ohm)                1.453914e+02  2.200384e+01     1.513421e+01
Rho_0 (Ohm m)          

c:\Users\andba122\AppData\Local\anaconda3\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\andba122\AppData\Local\anaconda3\Lib\site-packages\altair\vegalite\v5\api.py:362: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(
c:\Users\andba122\AppData\Local\anaconda3\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\andba122\AppData\Local\anaconda3\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\andba122\AppData\Local\anaconda3\Lib\site-packages\altair\vegalite\v5\api.py:362: AltairDep

alt.HConcatChart(...)

Chi-squared w DOF: 1.2101868071914106
DOF: 52
Chi-squared divided by DOF: 0.023272823215219434
(This seems to match IVIUM's reduced chi-squared value in order of magnitude, but their DOF does not take into account the number of fit parameters)
Chi-squared not normalised w DOF: 2972036.5900460705
Chi-squared not normalised divided by DOF: 57154.54980857828

Exportable table of circuit 11 fit results:
                   Fitted Value  Uncertainty  Uncertainty (%)
Circuit 11       R0-CustomR1-W1          NaN              NaN
Freqmin (Hz)               0.01          NaN              NaN
Freqmax (Hz)               6000          NaN              NaN
Chi-squared            1.210187          NaN              NaN
Chi-squared/DOF        0.023273          NaN              NaN 

                        Fitted Value   Uncertainty  Uncertainty (%)
R0 (Ohm)                1.120737e+02  5.127843e+00     4.575419e+00
Rho_0 (Ohm m)           1.886739e+08  9.222625e-03     4.888130e-09
lambda_param (m)   

In [143]:
# Simulate the circuit using initial guess values for the circuit parameters across the full range of frequencies

# TEST THIS
 
circuit_to_simulate = 11  # Choose the circuit index to simulate   2 = "R0-CustomR1"  3 = "R0-CustomR1-W1"
initial_guess_to_simulate = initial_guesses[circuit_to_simulate]
initial_guess_to_simulate = [130, 2e8, 4e-10, 7.9686e-11, 40e-10,  1000]
# ("R0 (Ohm), Rho_0 (Ohm m), lambda_param (m), epsilonepsilon_0 (F/m), delta (m)", W )

# circuit_to_simulate = 2  # Choose the circuit index to simulate   2 = "R0-CustomR1"  3 = "R0-CustomR1-W1"
# initial_guess_to_simulate = initial_guesses[circuit_to_simulate]
# initial_guess_to_simulate = [130, 1e10, 8e-10, 7.9686e-11, 4e-9,  ]
# # ("R0 (Ohm), Rho_0 (Ohm m), lambda_param (m), epsilonepsilon_0 (F/m), delta (m)", W )

# Create the circuit with the initial guess values
circuit = CustomCircuit(equivalence_circuits[circuit_to_simulate], initial_guess=initial_guess_to_simulate)

# Predict the impedance using the initial guess values
Z_simulated = circuit.predict(frequencies_orig)

#bounds_selected = bounds_defaults[circuit_to_simulate]
bounds_selected = ([50, 0, 0, 7.9685e-11, 0, 0],                           # Lower bounds
# [np.inf, np.inf, np.inf, 7.9687e-11, (film_initial_thickness*5), np.inf ], )   # Upper bounds
[np.inf, np.inf, np.inf, 7.9687e-11, np.inf, np.inf ], )   # Upper bounds


# ("R0 (Ohm), Rho_0 (Ohm m), lambda_param (m), epsilonepsilon_0 (F/m), delta (m)")

circuit.fit(frequencies, Z,weight_by_modulus=True, bounds = bounds_selected)                    #bounds=bounds, etc.
Z_fit_full = circuit.predict(frequencies_orig)

# Plot the simulated data using the built-in plot method from impedance.py
# chart = circuit.plot(f_data=frequencies_orig, Z_data=Z_simulated)

chart = circuit.plot(f_data=frequencies_orig, Z_data=Z_fit_full)
chart = circuit.plot(f_data=frequencies, Z_data=Z)
print("Circuit",circuit_to_simulate,"fitting:")
#print(circuit.parameters_)
#print("\nConfidence intervals for the parameters:")
#print(circuit.conf_)
fit_results = circuit.parameters_
fit_results_uncert = circuit.conf_    # One standard deviation error estimates for fit parameters
fit_results_uncertpct = 100 * fit_results_uncert / fit_results
#print("\nConfidence intervals as percentages:")
#print(fit_results_uncertpct)

# Create a DataFrame to concatenate the results
labels = []
labels = (parameter_strings[circuit_to_simulate].split(", "))

data = {
    'Fitted Value': fit_results,
    'Uncertainty': fit_results_uncert,
    'Uncertainty (%)': fit_results_uncertpct
}
results_df = pd.DataFrame(data, index=labels)

# Print the formatted table like in the loop
print("\nExportable table of circuit", circuit_to_simulate, "fit results:")
print(results_df, "\n")

# Calculate residuals for both real and imaginary parts
residuals = Z - circuit.predict(frequencies)

# Normalize residuals by the impedance magnitudes
chi_squared_w_dof = np.sum(((residuals.real)**2 + (residuals.imag)**2) / 
                           ((Z.real)**2 + (Z.imag)**2))

# Divide by degrees of freedom (DOF)
dof = len(Z) - len(circuit.parameters_)

# Print the chi-squared value normalized by impedance magnitudes and divided by DOF
print(f"Chi-squared normalized by impedance magnitudes / DOF: {chi_squared_w_dof / dof:.6f}")

chart.display()


c:\Users\andba122\AppData\Local\anaconda3\Lib\site-packages\impedance\models\circuits\circuits.py:145: UserWarning: Simulating circuit based on initial parameters
  warnings.warn("Simulating circuit based on initial parameters")
c:\Users\andba122\AppData\Local\anaconda3\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\andba122\AppData\Local\anaconda3\Lib\site-packages\altair\vegalite\v5\api.py:362: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(
c:\Users\andba122\AppData\Local\anaconda3\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\andba122\AppData\Local\anaconda3\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWa

Circuit 11 fitting:

Exportable table of circuit 11 fit results:
                        Fitted Value   Uncertainty  Uncertainty (%)
R0 (Ohm)                9.215871e+01  3.963088e+00     4.300285e+00
Rho_0 (Ohm m)           9.999499e+07  5.247077e-04     5.247340e-10
lambda_param (m)        2.958657e-10  7.556656e-12     2.554083e+00
epsilonepsilon_0 (F/m)  7.968666e-11  3.747671e-12     4.703009e+00
delta (m)               5.719940e-08  0.000000e+00     0.000000e+00
W1 (Ohm/sqrt(s)         1.435280e+03  4.086084e+01     2.846890e+00 

Chi-squared normalized by impedance magnitudes / DOF: 0.012958


alt.HConcatChart(...)

In [144]:
# ChatGPT version for merging CSVs. Works reasonably.
#"""

import os
import glob
import pandas as pd
import numpy as np

# Get the Fitting folder path
fitting_folder = os.path.join(os.path.dirname(file_name), 'Fitting')

# Find all CSV files ending with 'pyfit_FitParamAllCircs.csv' in the Fitting folder
all_circuits_files = glob.glob(os.path.join(fitting_folder, '*pyfit_FitParamAllCircs.csv'))

# Sort files alphabetically to ensure consistency
all_circuits_files.sort()

# Initialize an empty list to store DataFrames
df_list = []

# Load each CSV and add filename as header
for file in all_circuits_files:
    # Read the CSV file
    df = pd.read_csv(file)
    
    # Get file name without path and extension
    file_prefix = os.path.basename(file).split('_pyfit')[0]
    
    # Create header row with filename
    header_df = pd.DataFrame([[file_prefix] + [''] * (len(df.columns)-1)], columns=df.columns)
    
    # Combine header with data
    df = pd.concat([header_df, df], ignore_index=True)
    
    # Add empty column as spacer (except for last file)
    if file != all_circuits_files[-1]:
        df['spacer'] = ''
    
    df_list.append(df)

# Concatenate all DataFrames horizontally
merged_df = pd.concat(df_list, axis=1, ignore_index=True)

# Save the merged DataFrame as a new CSV file
output_file = os.path.join(fitting_folder, '_Merged_FitParamAllCircs.csv')
merged_df.to_csv(output_file, index=False)

print(f"Successfully merged {len(all_circuits_files)} files into '{output_file}'.")

#"""

Successfully merged 12 files into 'Fitting\_Merged_FitParamAllCircs.csv'.


In [145]:
import os
import glob
import pandas as pd
import numpy as np

# Get the Fitting folder path
fitting_folder = os.path.join(os.path.dirname(file_name), 'Fitting')

# Find all CSV files ending with 'shortChiSqs.csv' in the Fitting folder
chi_sq_files = glob.glob(os.path.join(fitting_folder, '*_shortChiSqs.csv'))

# Sort files alphabetically to ensure consistency
chi_sq_files.sort()

# Initialize an empty list to store DataFrames
df_list = []

# Load each CSV and add filename as header
for file in chi_sq_files:
    # Read the CSV file
    df = pd.read_csv(file)
    
    # Get file name without path and extension (up to _shortChiSqs)
    file_prefix = os.path.basename(file).split('_shortChiSqs')[0]
    
    # Create header row with filename
    header_df = pd.DataFrame([[file_prefix] + [''] * (len(df.columns)-1)], 
                           columns=df.columns)
    
    # Combine header with data
    df = pd.concat([header_df, df], ignore_index=True)
    
    # Add empty column as spacer (except for last file)
    if file != chi_sq_files[-1]:
        df['spacer'] = ''
    
    df_list.append(df)

# Concatenate all DataFrames horizontally
merged_df = pd.concat(df_list, axis=1, ignore_index=True)

# Save the merged DataFrame as a new CSV file
output_file = os.path.join(fitting_folder, '_Merged_ShortChiSqs.csv')
merged_df.to_csv(output_file, index=False)

print(f"Successfully merged {len(chi_sq_files)} Chi-squared files into '{output_file}'.")

Successfully merged 13 Chi-squared files into 'Fitting\_Merged_ShortChiSqs.csv'.


In [146]:
#Combine exported CSVs into one CSV for all circuits and EIS scans. Cannot handle failure of fitting for a circuit.
"""

import os
import glob
import pandas as pd
import numpy as np

# Get the Fitting folder path
fitting_folder = os.path.join(os.path.dirname(file_name), 'Fitting')

# Find all files ending with 'pyfit_FitParamAllCircs.csv' in the Fitting folder
all_circuits_files = glob.glob(os.path.join(fitting_folder, '*pyfit_FitParamAllCircs.csv'))

# Sort the files to ensure consistent ordering
all_circuits_files.sort()

# Create an empty list to store the DataFrames
dfs = []

# Read each file and append to the list
for f in all_circuits_files:
    # Read CSV file
    df = pd.read_csv(f)
    
    # Get the file name without path and extension to use as column prefix
    file_prefix = os.path.basename(f).split('_pyfit')[0]
    
    # Set the Parameter column as index if it exists
    if 'Parameter' in df.columns:
        df.set_index('Parameter', inplace=True)
    
    # Make sure the index is unique
    if not df.index.is_unique:
        # Add a suffix to duplicate indices
        df.index = pd.Index([f"{idx}_{i}" if list(df.index).count(idx) > 1 else idx 
                           for i, idx in enumerate(df.index)])
    
    # Add empty column after each DataFrame
    empty_col = pd.DataFrame(np.nan, index=df.index, columns=[''])
    
    # Append both the data and empty column
    dfs.append(df)
    dfs.append(empty_col)

# Remove the last empty column (we don't need one after the final DataFrame)
dfs.pop()

# Concatenate all DataFrames horizontally
combined_df = pd.concat(dfs, axis=1)

# Save the combined DataFrame to a new CSV file
output_file = os.path.join(fitting_folder, '_Combined_AllCircuits.csv')
combined_df.to_csv(output_file)

print(f"Combined data saved to: {output_file}")

# Display the first few rows of the combined DataFrame
print("\nFirst few rows of the combined data:")
print(combined_df.head())

"""

'\n\nimport os\nimport glob\nimport pandas as pd\nimport numpy as np\n\n# Get the Fitting folder path\nfitting_folder = os.path.join(os.path.dirname(file_name), \'Fitting\')\n\n# Find all files ending with \'pyfit_FitParamAllCircs.csv\' in the Fitting folder\nall_circuits_files = glob.glob(os.path.join(fitting_folder, \'*pyfit_FitParamAllCircs.csv\'))\n\n# Sort the files to ensure consistent ordering\nall_circuits_files.sort()\n\n# Create an empty list to store the DataFrames\ndfs = []\n\n# Read each file and append to the list\nfor f in all_circuits_files:\n    # Read CSV file\n    df = pd.read_csv(f)\n    \n    # Get the file name without path and extension to use as column prefix\n    file_prefix = os.path.basename(f).split(\'_pyfit\')[0]\n    \n    # Set the Parameter column as index if it exists\n    if \'Parameter\' in df.columns:\n        df.set_index(\'Parameter\', inplace=True)\n    \n    # Make sure the index is unique\n    if not df.index.is_unique:\n        # Add a suffix t

PLOTLY PLOTTING CODE MOVED TO BU7 FILe